# 評判分析で文章のポジネガを判別しよう

機械学習を用いた評判分析における記念碑的論文( http://www.cs.cornell.edu/home/llee/papers/sentiment.pdf )と同様のセットアップで分析を行い、論文の精度を上回れるかチャレンジしてみましょう！

## 0.前準備

python versionの確認します。<br>
jupyter notebookではシェルコマンドの文頭に"!"をつけるとそのシェルコマンドをnotebook上で実行することができます。<br> 

In [1]:
!python --version

Python 2.7.10 :: Anaconda 2.3.0 (x86_64)


カレントディレクトリの確認とデータディレクトリの確認をします。<br>
osモジュールを使うことでOS依存の機能を使えるようになります。

In [2]:
import os

In [3]:
print( os.listdir(os.path.normpath("./")) )

['.ipynb_checkpoints', 'data', 'README.md', '評判分析入門_advanced.ipynb', '評判分析入門_normal.ipynb']


In [4]:
print( os.listdir(os.path.normpath("./data/")) )

['mix20_rand700_tokens.zip', 'README', 'tokens']


## 1.dataの読み込みとモジュールのインポート

pyenvなどを用いているとpandasなどがimportできない場合があります。<br>
その可能性の１つとしてlocale（国毎に異なる単位）の設定不足があり得るので、ここではそれを明示的に操作します。<br>

In [5]:
def set_locale():
    default = os.environ.get('LC_ALL')
    print( "Your default locale is", default )
    if default is None:
        os.environ.setdefault('LC_ALL', 'ja_JP.UTF-8')
        print( "Your locale is set as ja_JP.UTF-8" )

set_locale()

Your default locale is None
Your locale is set as ja_JP.UTF-8


今回使うデータファイルのパスをpythonのリストとして取得します。<br>
globはパス名を見つけたりparseしたりするモジュールです( http://docs.python.jp/3/library/glob.html )。<br>
今回扱うデータは https://www.cs.cornell.edu/people/pabo/movie-review-data/ より取得しています。<br>
データ構造は下記のようになっています。<br>
- data
    - README
    - tokens
        - neg
            - file1.txt
            - file2.txt
            - ...
        - pos
            - file1.txt
            - file2.txt
            - ...

In [6]:
import glob

neg_files = glob.glob( os.path.normpath("./data/tokens/neg/*") )
pos_files = glob.glob( os.path.normpath("./data/tokens/pos/*") )

取得したファイルパスの確認。

In [7]:
print(neg_files[0:2])
print(pos_files[0:2])

['data/tokens/neg/cv000_tok-9611.txt', 'data/tokens/neg/cv001_tok-19324.txt']
['data/tokens/pos/cv000_tok-11609.txt', 'data/tokens/pos/cv001_tok-10180.txt']


データ読み込みのテストをします。

実際に文章を１つ読み込んでみて正しく読み込めているかを確認します。<br>
本データは１つのファイルに１文で映画のレビュー文章が記載されています。<br>
テキストの読み込みはエンコーディングの問題などでエラーが生じやすいので、慣れるまでは根気強くdebugしましょう。<br>

無事に読み込めたら、具体的に１つファイルの中身を読み込んで内容を確認してみましょう。<br>
sys はファイルサイズ取得などのシステム上の操作を行うモジュールです( http://docs.python.jp/3/library/sys.html )。

In [8]:
import sys

def text_reader(file_path):
    python_version = sys.version_info.major
    
    if python_version >= 3:
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                print(line)
    else:
        with open(file_path, 'r') as f:
            for line in f:
                print(line)

In [9]:
text_reader(neg_files[11])

* ( out of 4 = poor ) 1995 , g , 90 minutes [1 hour , 30 minutes] [comedy] starring : michael roescher ( hank royce ) , kristy young ( jinnie sue macallister ) , justin garms ( voice of gordy ) , james donadio ( gilbert sipes ) , written by leslie stevens , jay sommers , dick chevillat , produced by sybil robson , directed by mark lewis . " gordy " is not a movie , it is a 90-minute-long " sesame street " skit , and a very bad one at that . this movie is so stupid and dumb that it's depressing to think that some hollywood executives actually gave this the green light , and even more surprising is the fact that this is a disney movie . i'm sure children are the target audience of this movie , but only kids under the age of five may be able to tolerate it . it is the story of a farm a piglet named gordy ( voiced by garms ) , whose family has been taken away to " up north , " which we know means death . of course we can hear the animals talk to each other , and they actually went to the t

今回使うモジュールの情報をまとめておきます。<br>
詳細な中身などに関してはご自身で調べてみてください。<br>
- matplotlib : グラフなどを描写する<br>
http://matplotlib.org/
- pandas : dataframeでデータを扱い、集計や統計量算出などがすぐ実行できる<br>
http://pandas.pydata.org/
- collections : pythonで扱えるデータの型を提供する<br>
http://docs.python.jp/3/library/collections.html
- numpy : 行列などの数学的オブジェクトを扱う<br>
http://www.numpy.org/
- sklearn.feature_extraction.DictVectorizer : 辞書データをベクトルの形に変換する<br>
http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.DictVectorizer.html
- sklearnのモデル : SVM, NB, RF<br>
http://scikit-learn.org/stable/tutorial/basic/tutorial.html
- sklearn.grid_search : パラメタの最適な組み合わせ見つける<br>
http://scikit-learn.org/stable/modules/grid_search.html

In [10]:
import matplotlib # not used in this notebook
import pandas as pd # not used in this notebook

import collections
import numpy as np

from sklearn.feature_extraction import DictVectorizer

from sklearn import svm, naive_bayes
from sklearn.ensemble import RandomForestClassifier

from sklearn import grid_search

## 2.特徴ベクトルの作成

unigramを作成する関数を定義します。<br>
スペース区切りで単語を抽出し、その数をカウントする単純な関数となります。<br>

In [11]:
def word_counter(string):
    words = string.strip().split()
    count_dict = collections.Counter(words)
    return dict(count_dict)

def get_unigram(file_path):
    result = []
    python_version = sys.version_info.major
    
    if python_version >= 3:
        for file in file_path:
            with open(file, 'r', encoding='utf-8') as f:
                for line in f:
                    count_dict = word_counter(line)
                    result.append(count_dict)
    else:
        for file in file_path:
            with open(file, 'r') as f:
                for line in f:
                    count_dict = word_counter(line)
                    result.append(count_dict)
    
    return result

関数の挙動を確認してみましょう。

In [12]:
word_counter("I am YK. I love data analysis using python.")

{'I': 2,
 'YK.': 1,
 'am': 1,
 'analysis': 1,
 'data': 1,
 'love': 1,
 'python.': 1,
 'using': 1}

この関数を用いて、negative と positive 両方で unigram を作成します。<br>
得られた2つのリストを合わせてモデルのインプット（説明変数）とします。
リストの結合は "+" で実施できます。 ex.) [1] + [2] = [1,2]<br>
negative と positive は各700文ずつありますが、そのうちいくつを使うかをここで指定します。初期設定では全てのデータを使うことになっていますが、後の過程で memory 不足になるようでしたらこの数を減らしてください。<br>

また、 jupyter notebook では %% をつけることで magic commands ( https://ipython.org/ipython-doc/3/interactive/magics.html ) という便利なコマンドを実行できます。ここでは処理にかかる時間をセルに表示するコマンドを使用しています。

In [13]:
%%time

DATA_NUM = 700

unigrams_data = get_unigram(neg_files[:DATA_NUM]) + get_unigram(pos_files[:DATA_NUM])

CPU times: user 390 ms, sys: 120 ms, total: 510 ms
Wall time: 836 ms


得られたunigram_dataを確認してみます。単語の出現数がカウントされていることが確認できます。<br>
合わせてそのデータサイズも確認してみます。

In [14]:
print( unigrams_data[0] )
print( "data size :", sys.getsizeof(unigrams_data) / 1000000, "[MB]" )

{'park': 1, 'yaz': 3, 'each': 1, 'boy': 1, 'and': 17, 'jack': 1, 'out': 3, ')': 6, 'climax': 1, 'counter-terrorist': 1, 'he': 4, 'soldiers': 1, 'extremely': 1, 'guilty': 1, 'feels': 1, "it's": 2, 'e-mail': 1, 'vibrant': 1, 'leaves': 1, 'segment': 1, 'rub': 1, 'hollywood': 1, 'href=': 2, 'mass': 1, 'result': 2, 'requires': 1, 'than': 1, 'gangster': 1, 'up': 4, 'else': 1, 'quinn': 5, 'is': 9, 'superstar': 1, 'lite': 1, 'well': 1, 'wife': 1, 'language': 1, 'rourke': 2, 'genre': 1, 'as': 1, 'ex-cia': 1, 'camera': 1, 'tsui': 2, 'much': 4, 'finds': 1, 'an': 5, 'looks': 1, 'jean-claude': 2, 'shirtless': 1, '>http': 1, 'motorcycle': 1, '"': 22, 'amusement': 1, '.': 31, 'mention': 1, 'opponent': 1, '//www': 2, '>jpeck1@gl': 1, 'wacky': 1, 'mones': 1, 'fits': 1, 'persona': 1, 'us': 2, '30': 1, 'role': 2, 'work': 2, 'them': 2, 'van': 4, 'rodman': 7, "stavros'": 1, 'mercenary': 1, 'explosions': 1, 'timecop': 1, 'peck': 1, 'just': 3, 'job': 1, 'but': 4, 'monks': 1, 'to': 20, 'blatantly': 1, 'they':

上で得られたデータは unigram という 1400 の要素を持つリストであり、各要素は key と value からなる辞書となっています。<br>
これを扱いやすい行列の形にします。<br>
ここでは各行が１つのレビューテキストに対応するようにして、各列が単語、要素がその単語の出現数というデータを作成します。<br>
scikit-learn で実装されている DictVectorizer という関数を使うことでそれが簡単に実行できます。<br>

In [15]:
%%time
vec = DictVectorizer()
feature_vectors_csr = vec.fit_transform( unigrams_data )

CPU times: user 724 ms, sys: 33.7 ms, total: 758 ms
Wall time: 774 ms


作成したデータを確認してみます。

In [16]:
feature_vectors_csr

<1400x44219 sparse matrix of type '<class 'numpy.float64'>'
	with 496525 stored elements in Compressed Sparse Row format>

行列の全成分（行成分×列成分）は 60,000,000 要素くらいありますが、このうちのほとんどは 0 で 0 以外の値が入っているのは500,000程度です。<br>
この CSR(Compressed Sparse Row) matrix というのはこのような疎行列をの 0 でない成分だけを保持する賢いものになっています。<br>

一方で 0 の成分を陽に保って普通の行列としてデータを保持することも可能です。<br>

In [17]:
feature_vectors = vec.fit_transform( unigrams_data ).toarray()
print( "data dimension :", feature_vectors.shape )
print( feature_vectors[0] )
print( "data size :", sys.getsizeof(feature_vectors) / 1000000, "[MB]" )

data dimension : (1400, 44219)
[  0.   0.  22. ...,   0.   0.   0.]
data size : 495.252912 [MB]


こちらはデータが非常に大きくなっていますが、これは 0 という成分を陽に保持しているためです。<br>
この段階で memory error が生じる場合は一度 kernel を restart して DATA_NUM の数を減らして再実行してください。<br>

## 3.ラベルデータの作成

今回扱うデータセットは全てに negative, positive というラベルが振られています。<br>
ここではそのラベルを neagtive → 0, neagtive → 1 とすることで二値判別問題のセットアップを構築します。<br>
先ほど作った説明変数となる特徴ベクトルはnegative sample 700文とpositive sample 700文を縦につなげて作ったものなので、0が700個と1が700個並んでいるベクトルを作成すれば必要なラベルを作れます。<br>

In [18]:
labels = np.r_[np.tile(0, DATA_NUM), np.tile(1, DATA_NUM)]

正しい位置で0と1の振替がなされているか確認します。

In [19]:
print( labels[0], labels[DATA_NUM-1], labels[DATA_NUM], labels[2*DATA_NUM-1]  )

0 0 1 1


## 4.学習用データとテスト用データの作成方法

論文の記述によれば、データを偏りがないように3分割に分け、 three fold cross validation でモデルを評価しています。<br>
ここでは乱数を生成して、データを3等分することで同様の状況を再現することにします。<br>
結果の再現性を担保するために乱数の seed も設定しておきます。<br>

In [20]:
np.random.seed(7789)

shuffle_order = np.random.choice( 2*DATA_NUM, 2*DATA_NUM, replace=False )

生成した乱数の中身を確認します。

In [21]:
print( "length :", len(shuffle_order) )
print( "first 10 elements :", shuffle_order[0:10] )

length : 1400
first 10 elements : [1235 1232  910  162  343 1160  221  545 1112 1322]


分割したデータセットに含まれるラベル=1の数を数えることでデータの偏りが生じていないかを確認します。<br>
明らかに偏りが生じてしまった場合は乱数のseedを設定し直します。<br>

In [22]:
one_third_size = int( 2*DATA_NUM / 3. )
print( "one third of the length :", one_third_size )

print( "# of '1' in 1st set :", np.sum( labels[ shuffle_order[:one_third_size] ]  ) )
print( "# of '1' in 2nd set :", np.sum( labels[ shuffle_order[one_third_size:2*one_third_size] ]  ) )
print( "# of '1' in 3rd set :", np.sum( labels[ shuffle_order[2*one_third_size:] ]  ) )

one third of the length : 466
# of '1' in 1st set : 227
# of '1' in 2nd set : 233
# of '1' in 3rd set : 240


## 5.モデルを学習して精度を検証

学習に必要な関数を定義します。<br>
ここではモデルとして{Support Vector Machine(SVM), Naive Bayes(NB), Random Forest(RF)}を用います。<br>
モデルの性能測定は予測と答えが一致する数をカウントして正答率を求めることで実施します。<br>

与えられたリストをN分割する関数を定義します。<br>
割り切れない場合はうしろのリストに格納します。<br>

In [23]:
def N_splitter(seq, N):
    avg = len(seq) / float(N)
    out = []
    last = 0.0
    
    while last < len(seq):
        out.append( seq[int(last):int(last + avg)] )
        last += avg
        
    return np.array(out)

望ましい動作をするか確認してみます。

In [24]:
N_splitter(range(14), 3)

array([range(0, 4), range(4, 9), range(9, 14)], dtype=object)

モデルの学習や予測のための関数を定義します。<br>
- train_model : 説明変数とラベルと手法を与えることでモデルを学習する
- predict : モデルと説明変数を与えることでラベルを予測する
- evaluate_model : 予測したラベルと実際の答えの合致数を調べる
- cross_validate : cross_validationを実行する

In [25]:
def train_model(features, labels, method='SVM', parameters=None):
    ### set the model
    if method == 'SVM':
        model = svm.SVC()
    elif method == 'NB':
        model = naive_bayes.GaussianNB()
    elif method == 'RF':
        model = RandomForestClassifier()
    else:
        print("Set method as SVM (for Support vector machine), NB (for Naive Bayes) or RF (Random Forest)")
    ### set parameters if exists
    if parameters:
        model.set_params(**parameters)
    ### train the model
    model.fit( features, labels )
    ### return the trained model
    return model

def predict(model, features):
    predictions = model.predict( features )
    return predictions

def evaluate_model(predictions, labels):
    data_num = len(labels)
    correct_num = np.sum( predictions == labels )
    return data_num, correct_num

def cross_validate(n_folds, feature_vectors, labels, shuffle_order, method='SVM', parameters=None):
    result_test_num = []
    result_correct_num = []
    
    n_splits = N_splitter( range(2*DATA_NUM), n_folds )

    for i in range(n_folds):
        print( "Executing {0}th set...".format(i+1) )
        
        test_elems = shuffle_order[ n_splits[i] ]
        train_elems = np.array([])
        train_set = n_splits[ np.arange(n_folds) !=i ]
        for j in train_set:
            train_elems = np.r_[ train_elems, shuffle_order[j] ]
        train_elems = train_elems.astype(np.integer)

        # train
        model = train_model( feature_vectors[train_elems], labels[train_elems], method, parameters )
        # predict
        predictions = predict( model, feature_vectors[test_elems] )
        # evaluate
        test_num, correct_num = evaluate_model( predictions, labels[test_elems] )
        result_test_num.append( test_num )
        result_correct_num.append( correct_num )
    
    return result_test_num, result_correct_num

上記関数はCross validationの数を変数として設定できるように作ってあります。<br>
今回は上述のように 3-folds で分析を行います。<br>

In [26]:
N_FOLDS = 3

準備ができたのでここでモデルを学習してその精度を確認してみましょう。<br>
とりあえず何も考えずに準備した Bag Of Words をインプットにしてモデルを学習し、その精度を確認してみます。<br>

In [27]:
%%time
ans,corr = cross_validate(N_FOLDS, feature_vectors_csr, labels, shuffle_order, method='SVM', parameters=None)

Executing 1th set...
Executing 2th set...
Executing 3th set...
CPU times: user 13.8 s, sys: 239 ms, total: 14.1 s
Wall time: 15 s


結果を確認します。

In [28]:
print( "average precision : ", np.around( 100.*sum(corr)/sum(ans), decimals=1 ), "%" )

average precision :  63.2 %


実際にどのような文章では予測が正解してどのような文章では予測を外しているのかを見てみます。<br>
モデルを学習して予測を行い、データを見てみます。<br>

In [29]:
%%time

svm_model = train_model(
    features=feature_vectors_csr[shuffle_order[0:950],:]
    , labels=labels[shuffle_order[0:950]]
    , method='SVM'
    , parameters=None
)

CPU times: user 3.43 s, sys: 62.1 ms, total: 3.49 s
Wall time: 4.17 s


In [30]:
print("data : " ,shuffle_order[970:980], "correct label : ", labels[shuffle_order[970:980]])
print( "predict label : ", predict(svm_model, feature_vectors_csr[shuffle_order[970:980], :]) )

data :  [ 490 1276  794  892  504  463   48  289 1218 1137] correct label :  [0 1 1 1 0 0 0 0 1 1]
predict label :  [1 1 0 0 0 1 0 1 1 0]


予測が間違っているものを見てみます。<br>

In [31]:
text_reader(neg_files[490-1])

* directed and co-produced by jodie foster . written by w . d . richter . cast : holly hunter , robert downey , jr . , anne bancroft , dylan mcdermott , charles durning , geraldine chaplin , steve guttenberg , claire danes , cynthia stevenson . distributed by paramount and polygram . the title " home for the holidays " suggests either a feel-good story , a family drama or a comedy , but guesses can be wrong . there was a made-for-tv movie with that title in 1972 which turned out to be a christmastime thriller . now , the current film's distributors have announced it as a comedy , but this description will do only if you stretch it . claudia ( holly hunter ) just lost her job as an art restorer at a museum . ( you wait a long time to find out that the museum is in chicago . ) with some trepidation , claudia leaves behind her sixteen-year old daughter ( we later learn in passing that hunter is unmarried ) and flies to another state to spend thanksgiving with her parents . ( you wait a lo

長いので全部読むのは大変ですが、読んでみると確かに negative なレビューだとわかります。<br>
一方で単語だけ見ると　feel-good や happy など positive と捉えられるような単語も出現しています。<br>

予測が当たっているものも見てみます。<br>
ファイルの指定方法に注意してください。

In [32]:
text_reader(pos_files[1276-700-1])

in these days of overlong movies ( meet joe black , the thin red line , the mask of zorro ) it is a shame that films like waking ned devine can't be longer than a paltry 90 minutes . this is just a cute movie , even through its mildly risque subject matter . old friends jackie ( bannen ) and michael ( kelley ) try to find the lottery winner ( they deduce must live in their dinky town of about 60 ) so that they might kiss up to him and share the winnings . through process of elimination , they find that it must be lovable old ned devine , who they find sitting in front of his tv , clutching the winning lottery ticket in his cold dead hand . what results is thuroughly amusing , as jackie tries to convince his wife that not claiming it would be wrong , and that they could really benefit . after all , old ned won't miss it . rather than divulge the later twists and turns , i'll stop here merely pointing out that jackie and michael get into all sorts of trouble in their little sleepy irish 

最後の一文などで確かに positive な評価をしているレビューだということが見て取れます。

## 6.パラメタチューニング

grid searchでパラメタをチューニングすることで、どの程度精度が上がるかを確認してみます。<br>
scikit-learnにはgrid searchが実装されているので、ここではそれを用いてパラメタをチューニングしてみます。<br>
grid search に関しては http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html など。<br>
下記セルの実行には4,5分程度かかります。

In [33]:
%%time

search_parameters = [
    {'kernel': ['rbf'], 'gamma': [1e-2, 1e-3, 1e-4], 'C': [0.1, 1, 10, 100, 1000]},
    {'kernel': ['linear'], 'C': [0.1, 1, 10, 100, 1000]}
]

model = svm.SVC()
clf = grid_search.GridSearchCV(model, search_parameters)
clf.fit( feature_vectors_csr, labels )

CPU times: user 4min 36s, sys: 2.85 s, total: 4min 39s
Wall time: 4min 45s


grid searchによって発見したパラメタやスコアを確認してみます。

In [34]:
print("best paremters : ", clf.best_params_)
print("best scores : ", clf.best_score_)

best paremters :  {'gamma': 0.0001, 'C': 100, 'kernel': 'rbf'}
best scores :  0.797857142857


In [35]:
%%time
ans,corr = cross_validate(N_FOLDS, feature_vectors_csr, labels, shuffle_order, method='SVM', parameters=clf.best_params_)

Executing 1th set...
Executing 2th set...
Executing 3th set...
CPU times: user 13.3 s, sys: 113 ms, total: 13.5 s
Wall time: 13.6 s


In [36]:
print( "average precision : ", np.around( 100.*sum(corr)/sum(ans), decimals=1 ), "%" )

average precision :  79.5 %


精度が15%程度も向上しました！機械学習においてモデルのパラメタチューニングが非常に重要であることが伺えます。

## 7.簡単な特徴量変換による効果の確認

論文に記載してあるように、Bag of Words(BoW) のカウント数を全て1にしてみることで精度にどのような変化が生じるかを調べてみます。

In [37]:
feature_vectors_csr.data[ feature_vectors_csr.data > 0 ] = 1.

変換したデータを用いて同様に学習プロセスを実行してみましょう。

In [38]:
%%time
ans, corr = cross_validate(N_FOLDS, feature_vectors_csr, labels, shuffle_order)

Executing 1th set...
Executing 2th set...
Executing 3th set...
CPU times: user 14.4 s, sys: 118 ms, total: 14.6 s
Wall time: 14.8 s


In [39]:
print( "average precision : ", np.around( 100.*sum(corr)/sum(ans), decimals=1 ), "%" )

average precision :  49.1 %


なんと精度がだいぶ落ちてしまいました。。。<br>
しかも現在の問題設定は 0 か 1 を判別するものなので、ランダムに判別するモデルを作ったとしても 50% 程度になります。<br>
それと同程度ということは、そもそもモデルの学習が上手くいっていないのではないかということが疑われます。<br>
そのことを検証してみるために、もう一度パラメタチューニングを実施してみます。<br>

In [40]:
%%time

search_parameters = [
    {'kernel': ['rbf'], 'gamma': [1e-2, 1e-3, 1e-4], 'C': [0.1, 1, 10, 100, 1000]},
    {'kernel': ['linear'], 'C': [0.1, 1, 10, 100, 1000]}
]

model = svm.SVC()
clf = grid_search.GridSearchCV(model, search_parameters)
clf.fit( feature_vectors_csr, labels )

CPU times: user 4min 40s, sys: 2.05 s, total: 4min 42s
Wall time: 4min 44s


In [41]:
print("best paremters : ", clf.best_params_)
print("best scores : ", clf.best_score_)

best paremters :  {'gamma': 0.001, 'C': 10, 'kernel': 'rbf'}
best scores :  0.814285714286


In [42]:
%%time
ans, corr = cross_validate(N_FOLDS, feature_vectors_csr, labels, shuffle_order, method='SVM', parameters=clf.best_params_)

Executing 1th set...
Executing 2th set...
Executing 3th set...
CPU times: user 14.1 s, sys: 107 ms, total: 14.2 s
Wall time: 14.4 s


In [43]:
print( "average precision : ", np.around( 100.*sum(corr)/sum(ans), decimals=1 ), "%" )

average precision :  82.7 %


パラメタを調整することで高い精度を発揮することが分かりました！<br>
この精度は論文に記載されているもの(82.9%)とほぼ同程度のものとなっています。<br>

## 8.SVM以外のモデルを実行

Naive Bayes は sparse matrix 型のインプットを受け付けないので、numpy arrayとして作成したデータを入れなければなりません。

In [44]:
%%time

ans, corr = cross_validate(N_FOLDS, feature_vectors, labels, shuffle_order, method='NB')
print( "average precision : ", np.around( 100.*sum(corr)/sum(ans), decimals=1 ), "%" )

Executing 1th set...
Executing 2th set...
Executing 3th set...
average precision :  62.3 %
CPU times: user 3.61 s, sys: 3.41 s, total: 7.02 s
Wall time: 7.68 s


Random Forest も実行してみます。

In [45]:
%%time

ans, corr = cross_validate(N_FOLDS, feature_vectors, labels, shuffle_order, method='RF')
print( "average precision : ", np.around( 100.*sum(corr)/sum(ans), decimals=1 ), "%" )

Executing 1th set...
Executing 2th set...
Executing 3th set...
average precision :  64.9 %
CPU times: user 2.37 s, sys: 841 ms, total: 3.22 s
Wall time: 3.24 s


gaussianNBはチューニング可能なパラメタはありませんが、RFはパラメタが多いため、興味がある方は下のセルのコメントアウトを外して、パラメタによって結果がどう変わるかを調べてみてください。

In [46]:
# %%time

# search_parameters = {
#     "max_depth": [3, 5, None],
#     "max_features": [1000,5000,10000,20000,30000],
#     "min_samples_split": [1,2,3,4,5,10],
#     "min_samples_leaf": [5,10,20,50],
#     "bootstrap": [True, False],
#     "criterion": ["gini", "entropy"]
# }

# model = RandomForestClassifier()
# clf = grid_search.GridSearchCV(model, search_parameters)
# clf.fit( feature_vectors_csr, labels )

# print(clf.best_params_)

# ans, corr = cross_validate(N_FOLDS, feature_vectors_csr, labels, shuffle_order, method='RF', parameters=clf.best_params_)
# print( "average precision : ", np.around( 100*sum(corr)/sum(ans), decimals=1 ), "%" )